In [77]:
import string
import numpy as np
import math
import random

In [78]:
text = '../documents_for_course/Robert_frost.txt'

We need 3 elements:
1st the vector with the initial probbilities
2nd the matrix for the second state
3rd the tensor containing the probabilities of a word given a previous word

By advice of the teacher we'll try first to use dictionaries: so pi will be a dictionary, A1 will be a dictionary of dictionaries and A2 will be a dictionary of dictionaries of dictionaries

In [79]:
input_texts = []

for line in open(text):
    line = line.rstrip().lower()
    if line:
        line = line.translate(str.maketrans('','',string.punctuation))
        input_texts.append(line)


In [80]:
word_to_index_mapping = {}
indice = 1

for text in input_texts:
    text = text.split()
    for word in text:
        if word not in word_to_index_mapping:
            word_to_index_mapping[word]=indice
            indice+=1

In [81]:
word_to_index_mapping

{'two': 1,
 'roads': 2,
 'diverged': 3,
 'in': 4,
 'a': 5,
 'yellow': 6,
 'wood': 7,
 'and': 8,
 'sorry': 9,
 'i': 10,
 'could': 11,
 'not': 12,
 'travel': 13,
 'both': 14,
 'be': 15,
 'one': 16,
 'traveler': 17,
 'long': 18,
 'stood': 19,
 'looked': 20,
 'down': 21,
 'as': 22,
 'far': 23,
 'to': 24,
 'where': 25,
 'it': 26,
 'bent': 27,
 'the': 28,
 'undergrowth': 29,
 'then': 30,
 'took': 31,
 'other': 32,
 'just': 33,
 'fair': 34,
 'having': 35,
 'perhaps': 36,
 'better': 37,
 'claim': 38,
 'because': 39,
 'was': 40,
 'grassy': 41,
 'wanted': 42,
 'wear': 43,
 'though': 44,
 'for': 45,
 'that': 46,
 'passing': 47,
 'there': 48,
 'had': 49,
 'worn': 50,
 'them': 51,
 'really': 52,
 'about': 53,
 'same': 54,
 'morning': 55,
 'equally': 56,
 'lay': 57,
 'leaves': 58,
 'no': 59,
 'step': 60,
 'trodden': 61,
 'black': 62,
 'oh': 63,
 'kept': 64,
 'first': 65,
 'another': 66,
 'day': 67,
 'yet': 68,
 'knowing': 69,
 'how': 70,
 'way': 71,
 'leads': 72,
 'on': 73,
 'doubted': 74,
 'if': 75

In [82]:
dataset_as_int = []

for line in input_texts:
    new_line = []
    line = line.split()
    for word in line:
        new_line.append(word_to_index_mapping[word])
    dataset_as_int.append(new_line)

In [83]:
dataset_as_int

[[1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14],
 [8, 15, 16, 17, 18, 10, 19],
 [8, 20, 21, 16, 22, 23, 22, 10, 11],
 [24, 25, 26, 27, 4, 28, 29],
 [30, 31, 28, 32, 22, 33, 22, 34],
 [8, 35, 36, 28, 37, 38],
 [39, 26, 40, 41, 8, 42, 43],
 [44, 22, 45, 46, 28, 47, 48],
 [49, 50, 51, 52, 53, 28, 54],
 [8, 14, 46, 55, 56, 57],
 [4, 58, 59, 60, 49, 61, 62],
 [63, 10, 64, 28, 65, 45, 66, 67],
 [68, 69, 70, 71, 72, 73, 24, 71],
 [10, 74, 75, 10, 76, 77, 78, 79],
 [10, 80, 15, 81, 82, 83, 5, 84],
 [85, 86, 8, 86, 87],
 [1, 2, 3, 4, 5, 7, 8, 10],
 [10, 31, 28, 16, 88, 89, 90],
 [8, 46, 91, 92, 93, 28, 94],
 [95, 96, 97, 98, 10, 99, 10, 100],
 [101, 102, 103, 4, 28, 104, 44],
 [105, 106, 12, 107, 108, 109, 110],
 [24, 111, 101, 96, 112, 113, 83, 114],
 [115, 116, 117, 118, 99, 26, 119],
 [24, 120, 121, 5, 122, 123],
 [124, 28, 96, 8, 125, 126],
 [28, 127, 128, 129, 28, 130],
 [105, 131, 101, 132, 133, 5, 134],
 [24, 135, 75, 48, 103, 136, 137],
 [28, 138, 32, 139, 28, 140],
 [129, 141, 142,

First we create the model

In [84]:
pi = {}
A1 = {}
A2 = {}

# Calculate pi probabilities
for line in input_texts:
    line = line.split()
    for index, word in enumerate(line):
        if index == 0 and word not in pi.keys():
            pi[word] = 1
        elif index == 0 and word in pi.keys():
            pi[word] += 1

for word_value in pi.keys():
    pi[word_value] = pi[word_value]/len(pi)


# Calculate A1 probabilities
for line in input_texts:
    line = line.split()
    for index, word in enumerate(line):
        if index == 1:
            if line[index-1] not in A1.keys():
                A1[line[index-1]] = {}
            if word not in A1[line[index-1]].keys():
                A1[line[index-1]][line[index]] = 0
            if word in A1[line[index-1]].keys():
                A1[line[index-1]][word] +=1
        else:
            pass
for state in A1.keys():
    total = len(A1[state])
    for word in A1[state]:
        A1[state][word] = A1[state][word]/total

# Calculate A2 probabilities
for line in input_texts:
    line = line.split()
    for index, word in enumerate(line):
        if index >= 2:
            if line[index-2] not in A2.keys():
                A2[line[index-2]] = {}
            if line[index-1] not in A2[line[index-2]].keys():
                A2[line[index-2]][line[index-1]] = {}
            if line[index] not in A2[line[index-2]][line[index-1]]:
                A2[line[index-2]][line[index-1]][word] = 1
            else:
                A2[line[index-2]][line[index-1]][word] += 1
        # else:
        #     pass
for state in A2.keys():
    total = len(A2)
    for second_word in A2[state].keys():
        second_total = len(A2[state])
        for third_word in A2[state][second_word].keys():
            A2[state][second_word][third_word] = A2[state][second_word][third_word] / total*second_total

In [85]:
def generate_line(pi,A1,A2):
    new_line = []
    max_pi = max([i for i in pi.values()])
    min_pi = min([i for i in pi.values()])
    limit = np.random.uniform(low=min_pi, high=max_pi)
    possible_words = [i for i in pi.keys()]
    np.random.shuffle(possible_words)
    for word in possible_words:
        if pi[word] <= limit:
            new_line.append(word)
            break
    
    for word in A1[new_line[0]].keys():
        if len(new_line) < 2:
            if A1[new_line[0]][word] <= limit:
                new_line.append(word)
                limit = limit + A1[new_line[0]][word]
                break
            else:
                for word in A1[new_line[0]]:
                    words_of_choice = [i for i in A1[new_line[0]]]
                    random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                    new_word = words_of_choice[random_word_selector]
                    limit = limit +  A1[new_line[0]][new_word]
                    new_line.append(new_word)
                    break

    previous_word = 1
    preprevious_word = 0                

    while len(new_line) < 6:
        if len(new_line) <= 2:
            for word in A2[new_line[preprevious_word]][new_line[previous_word]].keys():
                if A2[new_line[preprevious_word]][new_line[previous_word]][word] <= limit:
                    new_line.append(word)
                    limit = limit + A2[new_line[preprevious_word]][new_line[previous_word]][word]
                    previous_word +=1
                    preprevious_word +=1
                    break
                else:
                    for word in A2[new_line[preprevious_word]]:
                        words_of_choice = [i for i in A2[new_line[preprevious_word]]]
                        random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                        new_word = words_of_choice[random_word_selector]
                        for word in A2[new_line[preprevious_word]][new_word].keys():
                            random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                            new_new_word = words_of_choice[random_word_selector]
                            if A2[new_line[preprevious_word]][new_word][new_new_word] <= limit:
                                new_line.append(new_new_word)
                                limit = limit +  A2[new_line[preprevious_word]][new_word][new_new_word]
                                preprevious_word +=1
                                previous_word +=1
                                break
                        break
        else:
            try:
                if A2[new_line[preprevious_word]][new_line[previous_word]][word] <= limit:
                    new_line.append(word)
                    preprevious_word +=1
                    previous_word +=1
                    limit = limit + A2[new_line[preprevious_word]][new_line[previous_word]][word]
                    # break
            except:
                try:
                    for word in A2[new_line[preprevious_word]][new_line[previous_word]]:
                        words_of_choice = [i for i in A2[new_line[preprevious_word]][new_line[previous_word]]]
                        random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                        new_word = words_of_choice[random_word_selector]
                        if A2[new_line[preprevious_word]][new_line[previous_word]][new_word] <= limit:
                            limit = limit +  A2[new_line[preprevious_word]][new_line[previous_word]][new_word]
                            new_line.append(new_word)
                            preprevious_word +=1
                            previous_word +=1
                            break
                except:
                    try:
                        for word in A2[new_line[preprevious_word]]:
                            words_of_choice = [i for i in A2[new_line[preprevious_word]].keys()]
                            random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                            new_word = words_of_choice[random_word_selector]
                            for word in A2[new_line[preprevious_word]][new_word]:
                                words_of_choice = [i for i in A2[new_line[preprevious_word]][new_word].keys()]
                                random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                                new_new_word = words_of_choice[random_word_selector]
                                if A2[new_line[preprevious_word]][new_word][new_new_word] <= limit:
                                    limit = limit +  A2[new_line[preprevious_word]][new_word][new_new_word]
                                    new_line.append(new_new_word)
                                    preprevious_word +=1
                                    previous_word +=1
                                    break
                            break
                    except:
                        for word in A2.keys():
                            words_of_choice = [i for i in A2.keys()]
                            random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                            new_word = words_of_choice[random_word_selector]
                            for word in A2[new_word]:
                                words_of_choice = [i for i in A2[new_word].keys()]
                                random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                                new_new_word = words_of_choice[random_word_selector]
                                for word in A2[new_word][new_new_word].keys():
                                    words_of_choice = [i for i in A2[new_word][new_new_word].keys()]
                                    random_word_selector = np.random.randint(low=0, high=len(words_of_choice))
                                    new_new_new_word = words_of_choice[random_word_selector]
                                    if A2[new_word][new_new_word][new_new_new_word] <= limit:
                                        limit = limit +  A2[new_word][new_new_word][new_new_new_word]
                                        new_line.append(new_new_word)
                                        preprevious_word +=1
                                        previous_word +=1
                                        break
                                break
                            break
    return new_line    



In [86]:
def generate_stanza(line,pi,A1,A2):
    new_stanza = ""
    for line in range(4):
        line = generate_line(pi,A1,A2)
        correct_line = ' '.join(line)
        new_stanza = f"{new_stanza}\n{correct_line}"
    print(new_stanza)

In [91]:
generate_stanza(line,pi,A1,A2)


couldnt one talk to you to
around and under exclaimed spell saw
look here you are withdrawing into
proclaimed the time had come cottages


In [ ]:
# pi

In [ ]:
# A1

In [ ]:
# A2